# Qiskit Runtime REST API query & job submission

## Query API without authentication

In [52]:
import requests

url = 'https://us-east.quantum-computing.cloud.ibm.com/programs'
response = requests.get(url)

if response.status_code == 200:
    data = response.json()
else:
    print(f"Error: {response.status_code}")

In [53]:
[program['id'] for program in data['programs']]

['qasm3-runner', 'circuit-runner', 'sampler', 'estimator']

## Get temporary Access token from Auth API via API Token

In [54]:
with open('cloud_token') as file:
    token=file.read()
with open('crn_service') as file:
    crn_service = file.read()

In [55]:
import requests

url = 'https://iam.cloud.ibm.com/identity/token'
headers = {'Content-Type': 'application/x-www-form-urlencoded'}
data='grant_type=urn:ibm:params:oauth:grant-type:apikey&apikey='+token
auth_response = requests.post(url, headers= headers, data=data)
auth_id=auth_response.json()['access_token']
auth_type = auth_response.json()['token_type']

## GET available backends

In [56]:
url_backends = 'https://us-east.quantum-computing.cloud.ibm.com/backends'
headers = { 'Content-Type': 'application/json',
            'Service-CRN': crn_service,
            'Authorization':auth_type + ' ' + auth_id}

backends_response = requests.get(url_backends, headers=headers)
backends_response.json()

{'devices': ['ibm_algiers',
  'ibm_brisbane',
  'ibmq_qasm_simulator',
  'simulator_stabilizer',
  'simulator_mps',
  'simulator_extended_stabilizer',
  'simulator_statevector']}

In [57]:
backend = backends_response.json()['devices'][0]
backend

'ibm_algiers'

## Run a job

### set up qasm circuit

In [58]:
qasm_string='''
OPENQASM 3;
include "stdgates.inc";
qreg q[1];
creg c[1];
x q[0];
c[0] = measure q[0]; 
'''   

### run circuit via API

In [59]:
import requests

url = 'https://us-east.quantum-computing.cloud.ibm.com/jobs'

headers = { 'Content-Type': 'application/json',
            'Service-CRN': crn_service,
            'Authorization':auth_type + ' ' + auth_id,
            'x-qx-client-application': 'qiskit-version-2/0.39.2/'+'your_application' #specifying the application you might be running from. For an actual Integration project, this option it is invaluable to know where jobs are coming from. At this time the "qiskit-version-2/0.39.2/" string is a necessary prefix.
            }

job_input = {
    'program_id': 'sampler',
    "backend": backend, #Sessions work best with real backends (QCs)
    "start_session": True, #set to False if you just need to run a single job. When addressing simulators, comment out this line. 
    "params": {
        "circuits": [qasm_string]#*5
}}

response = requests.post(url, headers=headers, json=job_input)

if response.status_code == 200:
    job_id = response.json().get('id')
    print("Job created:",response.text)
else:
    print(f"Error: {response.status_code, response.text}")

Job created: {"id":"cl9pitd4nd4s6h1fo0vg","backend":"ibm_algiers","session_id":"cl9pitd4nd4s6h1fo0vg"}


### Run follow-up jobs in the same Session (optional)

In [60]:
if 'session_id' in response.json().keys():
    session_id=response.json()['session_id']
    print(session_id)
backend=response.json()['backend']
backend

cl9pitd4nd4s6h1fo0vg


'ibm_algiers'

In [61]:
job_input = {
    'program_id': 'sampler',
    "backend": backend,
    "params": {
        "circuits": [qasm_string]*5
}}

if 'session_id' in response.json().keys():
  job_input['session_id'] = session_id
  
response = requests.post(url, headers=headers, json=job_input)

if response.status_code == 200:
    job_id = response.json().get('id')
    print("Job created:",response.text)
else:
    print(f"Error: {response.status_code}")

Job created: {"id":"cl9pivt8f2v32dqep6cg","backend":"ibm_algiers","session_id":"cl9pitd4nd4s6h1fo0vg"}


### Query Websocket for live job update and close Session

It is very good practice to close a Session when all jobs are done. This will reduce wait time for subsequent Users. So we will listen for a status change of the last job via websocket and close the Session when done.

In [ ]:
import asyncio
import websockets

with open('token') as file:
    api_key=file.read()

async def receive_status_updates(api_key):
    uri = "wss://runtime-us-east.quantum-computing.ibm.com/stream/jobs/"+job_id 
    extra_headers = {'Authorization': f'Bearer {api_key}'}

    async with websockets.connect(uri, extra_headers=extra_headers) as websocket:
        while True:
            try:
                message = await websocket.recv()
                print(f"Received message: {message}")  
            except websockets.exceptions.ConnectionClosed:
                print("WebSocket connection closed.")
                break

await receive_status_updates(api_key)

#subsequently closing Session
status_result = requests.get(url+'/'+job_id,headers=headers)
sessionsurl = 'https://runtime-us-east.quantum-computing.ibm.com/sessions/'+session_id+'/close'

if status_result.json()['state']['status']=='Completed':
   response = requests.delete(sessionsurl,headers=headers)
else:
   print('waiting for job to complete before closing Session '+session_id)

if response.status_code==204:
    print('Session '+session_id+' was closed')

Received message: Listening to results for job: cn9q5sq1mh9g008yf980

WebSocket connection closed.
Session cn9q5dxpb7t0008r3wp0 was closed


## Wait for results

In [15]:
status_result = requests.get(url+'/'+job_id,headers=headers)
print(status_result.json()['state'],job_id)

{'status': 'Completed'} ck6qehjl5r7th0mrmk7g


### Get results

In [16]:
response_result= requests.get(url+'/'+job_id+'/results', headers=headers)

response_result.json()

{'quasi_dists': [{'1': 1.0}],
 'metadata': [{'shots': 1024,
   'readout_mitigation_overhead': 1.0,
   'readout_mitigation_time': 0.06403936599963345}]}

## Invalidate Token (Optional)

In [17]:
logout_url = 'https://us-east.quantum-computing.cloud.ibm.com/logout'

headers = { 'Content-Type': 'application/json',
            'Service-CRN': crn_service,
            'Authorization':auth_type + ' ' + auth_id}

logout_response = requests.post(logout_url, headers=headers)
print("response ok?:",logout_response.ok,logout_response.text)

response ok?: True 
